## Importing required packages

In [1]:
from pydantic import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

## Custom tool example

In [2]:
# using decorator to declare a function as tool
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

print(search.name)
print(search.description)
print(search.args)

search
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [3]:
class SearchInput(BaseModel):
    query: str = Field(description="Should be a search query")

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

print(search.name)
print(search.description)
print(search.args)

search-tool
Look up things online.
{'query': {'description': 'Should be a search query', 'title': 'Query', 'type': 'string'}}


In [4]:
def search_function(query: str):
    return "LangChain"

search = StructuredTool.from_function(
    func=search_function,
    name="Search",
    description="useful for when you need to answer question about current events"
)

print(search.name)
print(search.description)
print(search.args)

Search
useful for when you need to answer question about current events
{'query': {'title': 'Query', 'type': 'string'}}


In [5]:
result = search.invoke(input="What is langchain?")
print(result)

LangChain


## How to use with LLM agent

In [6]:
@tool
def get_word_length(word: str) -> int:
    """Returns the length of the word."""
    return len(word)


print(get_word_length.name)
print(get_word_length.description)
print(get_word_length.args)

get_word_length
Returns the length of the word.
{'word': {'title': 'Word', 'type': 'string'}}


In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# building chat prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an intelligent assistant. You are provided with a tool to calculate a word's length. Use these tool to answer question whenever you need."
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

### Binding tools and executing LLM agents

In [8]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents.format_scratchpad import format_to_tool_messages
from langchain.agents.output_parsers import ToolsAgentOutputParser
from langchain.agents import AgentExecutor

load_dotenv()


llm_with_tools = ChatGroq(model="mixtral-8x7b-32768").bind_tools(tools=[get_word_length])

# building an Agent
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_tool_messages(x["intermediate_steps"])
    }
    | prompt
    | llm_with_tools
    | ToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=[get_word_length], verbose=True)

In [9]:
list(agent_executor.stream(input={"input": "How many letters in the word 'Muntasir'?"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Muntasir'}`


8The word 'Muntasir' contains 8 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'Muntasir'}, log="\nInvoking: `get_word_length` with `{'word': 'Muntasir'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_n0ff', 'function': {'arguments': '{"word":"Muntasir"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-0117b461-02e8-4c89-b27b-829dd428ef57', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'Muntasir'}, 'id': 'call_n0ff', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1205, 'output_tokens': 94, 'total_tokens': 1299}, tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"Muntasir"}', 'id': 'call_n0ff', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_n0ff')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_n0ff', 'function': {'arguments': '{"word":"Muntasir"}', 'name': 'get_word_le